## About this workflow

Earth engine couldnt load collection when I first (1) filter date-get collection & then (2)filter geometry-get collection. Because step one was resulting with too many data. So reversing the steps worked. In this case we:

* Merge all Landsat collections (dataframe function)
* Mapbox setup & create "id" variables to be used in Dash
* HTML Function to return outputs from interactive tools
    * Map, Base Layer, Clicked Coordinate, Max Cloud, Date Range
* App Callbacks:
    * Return clicked marker (dl.Marker) & global lat lon variables
    * Return clciked coordinate (json.dumps(e))
    * Return base layer (url)
    * Return date
* First filter feature collection with geometry (coordinates & cloud)
* Then filter the resulting collection furhter with date range
* Calculate NDWI, simplify dataframe 
* PLot simplified dataframe

## Links

* Returning HTML in a functins before calling the app: https://github.com/mepearson/tacc_dash/blob/fd12ece9555fe88d76f4550679295d8a73406120/viz/models/leaflet/render.py

In [1]:
##-----------------------------------------##
#               Packages
##-----------------------------------------##
from datetime import datetime as dt
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import dash_table
import dash_leaflet as dl
from dash.dependencies import Output, Input

from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets

import ee, datetime
from datetime import date
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

#import eeconvert # Package to conver Earth Engine collection to dataframe or geodataframe
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


# Earth Engine Python API
ee.Initialize()



In [2]:
##-----------------------------------------##
#               Collection Attributes
##-----------------------------------------##


def makeLandsatSeries():

    lt4 = ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
    lt5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
    le7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    lc8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

    lt4 = lt4.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lt5 = lt5.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    le7 = le7.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lc8 = lc8.select(['B2','B3','B4','B5','B6','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])

    fullCollection = ee.ImageCollection(lt4.merge(lt5).merge(le7).merge(lc8))
    return fullCollection


##-----------------------------------------##
#               Mapbox Attributes
##-----------------------------------------##

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css',
'https://codepen.io/chriddyp/pen/brPBPO.css']


# Mapbox setup
mapbox_url = "https://api.mapbox.com/styles/v1/mapbox/{id}/tiles/{{z}}/{{x}}/{{y}}{{r}}?access_token={access_token}"
mapbox_token = 'pk.eyJ1IjoibmF0MSIsImEiOiJjazhpeDdwZ3gwM3FyM2RueTZwdDJ0bzB2In0.lqoQJb90lcn0cu-zulXWyw'
mapbox_ids = ["light-v9", "dark-v9", "streets-v9", "outdoors-v9", "satellite-streets-v9"]

# Element mapbox_ids
BASE_LAYER_ID = "base-layer-id"
BASE_LAYER_DROPDOWN_ID = "base-layer-drop-down-id"
MAP_ID = "map-id"
MARKER = "marker-id"
NEW_MARKER= "new-marker-id"
COORDINATE_CLICK_ID = "coordinate-click-id"


##-----------------------------------------##
#               Functions
##-----------------------------------------##

#Generate map functions
def render_marker():
    return [
        dl.Map(id=MAP_ID, style={'width': '1000px', 'height': '500px'},
               center=[-13, 48],
               zoom = 5,
               children=[
                   dl.TileLayer(id=BASE_LAYER_ID),
                   dl.Marker(id=MARKER, position=[56, 9.8], interactive=True,   opacity=0.8),
                   html.Div(id=NEW_MARKER)
                  
               ]),
        html.P("Coordinate (click on map):"),
        html.Div(id=COORDINATE_CLICK_ID),
        
        html.Div(
            [ 
                dcc.Input(id="cloud", type="number", placeholder="cloud",
                          min = 0, max = 100, step = 10,),
                #html.Hr(),
                html.Div(id="number-out"),
            ]),
        html.Div(
            [
                dcc.DatePickerRange(
                    id='my-date-picker-range',
                    #month_format='Do MMM, YYYY',
                    display_format="D-M-YYYY",
                    min_date_allowed=dt(1995,1,1),
                    max_date_allowed=dt.today(),
                    #initial_visible_month=dt(2017, 8, 5),
                    start_date=dt(2000,1,1),
                    #end_date=dt.today(),
                    start_date_placeholder_text="D-M-YYYY",
                    end_date_placeholder_text="D-M-YYYY",
                ),
                html.Div(id='output-container-date-picker-range')
            ])
    ]

def register_marker(app):
    
    # MARKER
    @app.callback(Output(NEW_MARKER, 'children'),
                  [Input(MAP_ID, 'click_lat_lng')])
    def new_marker(x):
        if x is not None:
            
            global lat
            global lon
            lat, lon = x
            print("Marker:", lat)
            
            return dl.Marker(position=[lat, lon])
        else:
            return None
    
    # COORDINATES
    @app.callback(Output(COORDINATE_CLICK_ID, 'children'),
                  [Input(MAP_ID, 'click_lat_lng')])
    def click_coord(e):
        if e is not None:
            print("Click:", json.dumps(e))
            return json.dumps(e)
        else:
            return "-"
    
    # BASE LAYER
    @app.callback(Output(BASE_LAYER_ID, "url"),
                  [Input(BASE_LAYER_DROPDOWN_ID, "value")])
    def set_baselayer(url):
        return url
    
    
    # CLOUD
    @app.callback(
    Output("number-out", "children"),
    [Input("cloud", "value")
    ],) 

    def number_render(cloud):


        if lat is not None:
            
            global latitude
            latitude = lat
        
            global longitude
            longitude = lon

            global Maxcloud
            Maxcloud = cloud 
            point = {'type':'Point', 'coordinates':[longitude, latitude]}; # This point is on turbid water
            
            #print("Cloud lat:",lat, "Type:", type(lat))
            #print("Cloud latitude:",latitude, "Type:", type(latitude))
            return ()
    
    # DATE
    @app.callback(
    dash.dependencies.Output('output-container-date-picker-range', 'children'),
    [dash.dependencies.Input('my-date-picker-range', 'start_date'),
     dash.dependencies.Input('my-date-picker-range', 'end_date')
    ])

    def update_output(start_date, end_date):
        
        # Define point
        point = {'type':'Point', 'coordinates':[longitude, latitude]}; # This point is on turbid water
        #print("Update Output Point", point)

        # Define Full Collection
        fullCollection= makeLandsatSeries()
        
        # Filter collection by Lat, Lon & Cloud
        filtered = fullCollection.filterBounds(ee.Geometry.Point(longitude,latitude)).filter(ee.Filter.lt('CLOUD_COVER', Maxcloud))

        #Count Size // Total number of images: 165
        count = int(filtered.size().getInfo())
        #print("Update Output Count:", count)

        #Info
        info = filtered.getRegion(point,500).getInfo()


        if start_date is not None:
            start_date = dt.strptime(start_date.split('T')[0], '%Y-%m-%d')
            start_date_string = start_date.strftime('%Y-%m-%d')
            startDate = start_date_string

            end_date = dt.strptime(end_date.split('T')[0], '%Y-%m-%d')
            end_date_string = end_date.strftime('%Y-%m-%d')
            endDate = end_date_string

            # Filter Collection with Lat, Lon & Cloud
            filtered2 = filtered.filterDate(startDate, endDate)

            #Count Size // Total number of images:
            count = int(filtered2.size().getInfo())
            print("Your inputs return", count, "images")

            info = filtered2.getRegion(point,500).getInfo()

            # Datframe
            df = pd.DataFrame(info,columns = ['id', 'longitude', 'latitude', 'time', 'blu', 'grn', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa'])
            #df = df.to_json(date_format='iso', orient='split')

            # Create an ID column with NAN values
            df['Satellite_ID'] = ('NAN')

            # Make sure that all NaN values are `np.nan` not `'NAN'` (strings)
            df = df.replace('NAN', np.nan)
            mask = df['id'].str.contains(r'LT04')
            df.loc[mask, 'Satellite_ID'] = ('L4')

            mask = df['id'].str.contains(r'LT05')
            df.loc[mask, 'Satellite_ID'] = ('L5')

            mask = df['id'].str.contains(r'LT05')
            df.loc[mask, 'Satellite_ID'] = ('L5')

            mask = df['id'].str.contains(r'LE07')
            df.loc[mask, 'Satellite_ID'] = ('L7')

            mask = df['id'].str.contains(r'LC08')
            df.loc[mask, 'Satellite_ID'] = ('L8')

            # Drop First row (a duplicate of column header)
            df = df.drop(df.index[0])

            # The Earth Engine time stamp in milliseconds since the UNIX epoch.
            # Link GEE: https://developers.google.com/earth-engine/glossary

            # Convert UNIX to datetime
            # Use pd.to_datetime() to convert unix epoch time
            # Result is like this: 1994-03-17 06:11:20.254
            df['time_stamp'] = pd.to_datetime(df['time'], unit='ms')

            # Convert to String inorder to apply string split
            df['time_string'] = df['time_stamp'].astype(str)

            # make the new date column using string indexing [0:10] will give us the year,month,date part
            # Result is like this 1994-03-17
            df['Date'] = df['time_string'].str[0:10]

            # Convert back to datetime
            df['Date'] = pd.to_datetime(df['Date'])

            # Drop unnecessary columns
            #df = df.drop('time_stamp', axis=1)
            df = df.drop('time_string', axis=1)

            # Sort by Date
            df = df.sort_values(by='Date')

            # Calculate NDWI
            #ndwi = (green - swir) / (green + swir)
            df['NDWI'] = (df['grn'] - df['swir1']) / (df['grn'] + df['swir1'])
            df['NDWI'] = pd.to_numeric(df['NDWI'])

            # Simplified Table for Dash
            simple_df = df[['Satellite_ID', 'longitude', 'latitude', 'NDWI', 'Date']].copy()


            #return fullCollection 
            print(simple_df)  

            # Create Line Chart
            fig = px.line(simple_df,
                          x = simple_df['Date'],
                          y = simple_df['NDWI'])

            # Fig Update & Set Title
            fig.update_layout(title_text="NDWI Values")
            fig.update_traces(overwrite=True)


            #return "start: {}, end: {}".format(start_date_string, end_date_string) 
            return [
                dcc.Graph(
                    id=column,
                    figure = fig
                )
                # check if column exists - user may have deleted it
                # If `column.deletable=False`, then you don't
                # need to do this check.
                for column in ["NDWI"] if column in simple_df
            ]


# Layout
def generate_layout():
    dlayout=html.Div([
        html.P('Satellite Image Analysis'),
        html.P("Select Location"),
        html.Div([
            html.P('Map Base Layer'),
            dcc.Dropdown(
                id=BASE_LAYER_DROPDOWN_ID,
                options=[{"label": i, "value": mapbox_url.format(id=i, access_token=mapbox_token)} for i in mapbox_ids],
                value=mapbox_url.format(id="light-v9", access_token=mapbox_token)
            ),
            html.P('Refine Date Range'),
            html.P('Opacity:')
        ],style={'float':'left','width':'20%','margin-right':'15px'}),
        html.Div(render_marker(),style={'float':'left','width':'75%'})
    ])
    
    return dlayout


##-----------------------------------------##
#               App
##-----------------------------------------##
app = dash.Dash(__name__, external_scripts=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Create layout.
app.layout = html.Div(generate_layout())

# Bind callbacks.
register_marker(app)


if __name__ == '__main__':
    app.run_server(debug=False, port=8150)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8150/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1586098345.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1586098345.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1586098345.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_9_0m1586098345.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_renderer/dash_renderer.v1_3_0m1586098345.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:23] "GET /_dash-component-suites/dash_leaflet/dash_leaflet.v0_0_3m1586264703.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:05:24] "G

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

127.0.0.1 - - [14/Apr/2020 09:05:24] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

...

127.0.0.1 - - [14/Apr/2020 09:06:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:06:04] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_9_0m1585774446.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:06:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:06:13] "POST /_dash-update-component HTTP/1.1" 200 -


Click:Marker: [-14.519780046326085, 47.29408264160157]
 -14.519780046326085


127.0.0.1 - - [14/Apr/2020 09:06:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 09:06:15] "POST /_dash-update-component HTTP/1.1" 200 -


Your inputs return 361 images
    Satellite_ID longitude latitude      NDWI       Date
95            L7   47.2941  -14.519  0.521127 2000-02-06
96            L7   47.2941  -14.519  0.420290 2000-03-25
1             L5   47.2941  -14.519  0.712707 2000-04-18
97            L7   47.2941  -14.519  0.765625 2000-04-26
2             L5   47.2941  -14.519  0.734597 2000-05-20
..           ...       ...      ...       ...        ...
333           L8   47.2941  -14.519  0.460396 2014-11-03
334           L8   47.2941  -14.519  0.337808 2014-11-19
309           L7   47.2941  -14.519  0.087344 2014-12-04
335           L8   47.2941  -14.519  0.554622 2014-12-05
218           L7   47.2941  -14.519  0.804598 2014-12-29

[361 rows x 5 columns]


127.0.0.1 - - [14/Apr/2020 09:06:29] "POST /_dash-update-component HTTP/1.1" 200 -
